In [ ]:
from pathlib import Path
desktop_path = Path.home() / "Desktop"
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, rf'{desktop_path}\RA\penicilin\Libaries')

import main as m


In [ ]:
from reload_lib import reload_lib as rl


In [1]:
import pandas as pd



In [4]:
init = m.main()
pen_cf_path = Path.joinpath(init.path_github, "data", r"penicillin_impact_factors.xlsx")

In [29]:

import re

def extract_unique_citations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # Regex to match \cite{...} and \pcite{...}
    pattern = r'\\(?:p)?cite\{(.*?)\}'
    matches = re.findall(pattern, text)

    # Split multiple citations within one command (e.g., \cite{ref1,ref2})
    all_citations = []
    for match in matches:
        all_citations.extend([entry.strip() for entry in match.split(',')])

    # Get unique entries
    unique_citations = list(set(all_citations))
    return unique_citations


In [ ]:
file_path = r"C:\Users\ruw\Desktop\RA\penicillin_results\manuscript.txt"

unique_citations = extract_unique_citations(file_path)
len(unique_citations)

66

In [41]:
for x in unique_citations:
    print(x)

ARUP2019HEALTHFOOTPRINT
Berenjian2019LearningTechnology
penVmedicinDK
Baehr2023LifePractices
healthcareLCA
Rowe2009HandbookExcipients
EuropeanCentreforDiseasePreventionandControlFactsResistance
Armstrong2022ChinaMarket
ecoinvent2023Ecoinvent3.9.1
emara2019
Kaasch2024EfficacyTrial
Mertz2009OutcomesWards
Steenmeijer2022TheAnalysis
PyPI2024Brightway22.4.7
Kasper2025LossesWaste
Lim2009British2009
nyberg2021
RADS2016BaggrundsnotatHospital
PharmaceuticalEnvironmentGroupPharmaTaskforce.
harding2018
Nelson1955TheProcess
RegionSjlland2022PneumoniAf
ISO2006ISO14044:2006enGuidelines
Deepak2022LifeImpacts
Perlman1950SomeProduction
Aboltins2015OralTrial
RIVM.DownloadsRIVM
Workman2024TopCountry
GreenhouseGasProtocolWorldDevelopment.
StaunstrupKLINISKRISIKOAFFALD
Hickey2019InternationalFootprints
AmgrosAmgros
Vidal2013OralPatients
WorldHealthOrganizationCommitmentsHealth
Bassani2022LifePackaging
Harding2008ALCA
Mohamud2024PostoperativeStudy
Li2019OralInfection
belboom2011
Harding2018GenericProduction

In [45]:
import re

def extract_relevant_bib_entries(bib_file_path, citation_keys):
    with open(bib_file_path, 'r', encoding='utf-8') as f:
        bib_content = f.read()

    # Regex to match full BibTeX entries
    entry_pattern = r'@[\w]+\{[^}]*\}'
    all_entries = re.findall(entry_pattern, bib_content, re.DOTALL)

    matched_entries = {}

    for entry in all_entries:
        # Extract the key from the entry
        key_match = re.match(r'@[\w]+\{([^,]+),', entry)
        if key_match:
            key = key_match.group(1).strip()
            if key in citation_keys:
                matched_entries[key] = entry

    return matched_entries

In [46]:
bib_file_path = r"C:\Users\ruw\Downloads\penicillin.bib"

matched_entries = extract_relevant_bib_entries(bib_file_path, unique_citations)
len(matched_entries)

66

In [ ]:

def save_bib_entries(matched_entries, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as f:
        for entry in matched_entries.values():
            f.write(entry.strip() + '\n\n')


In [50]:
bib_file_path_unq = r"C:\Users\ruw\Downloads\penicillin_unq.bib"
save_bib_entries(matched_entries, bib_file_path_unq)

In [52]:
import re

def extract_unique_citations(txt_file_path):
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # Match \cite{...} and \pcite{...}
    pattern = r'\\(?:p)?cite\{(.*?)\}'
    matches = re.findall(pattern, text)

    # Split multiple citations within one command
    all_citations = []
    for match in matches:
        all_citations.extend([entry.strip() for entry in match.split(',')])

    return set(all_citations)

def extract_relevant_bib_entries(bib_file_path, citation_keys):
    with open(bib_file_path, 'r', encoding='utf-8') as f:
        bib_content = f.read()

    # Match all BibTeX entries of any type
    entry_pattern = r'@[\w]+\{[^@]*?\n\}'
    all_entries = re.findall(entry_pattern, bib_content, re.DOTALL)

    matched_entries = {}
    for entry in all_entries:
        key_match = re.match(r'@[\w]+\{([^,]+),', entry)
        if key_match:
            key = key_match.group(1).strip()
            if key in citation_keys:
                matched_entries[key] = entry

    return matched_entries

def save_bib_entries(matched_entries, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as f:
        for entry in matched_entries.values():
            f.write(entry.strip() + '\n\n')

# Example usage
txt_file = r"C:\Users\ruw\Desktop\RA\penicillin_results\manuscript.txt"
bib_file = r"C:\Users\ruw\Downloads\penicillin.bib"
output_file = r"C:\Users\ruw\Downloads\penicillin_unq.bib"

citation_keys = extract_unique_citations(txt_file)
matched_entries = extract_relevant_bib_entries(bib_file, citation_keys)
save_bib_entries(matched_entries, output_file)


In [ ]:
with open(bib_file_path, 'r', encoding='utf-8') as f:
        bib_content = f.read()

entry_pattern = r'@[\w]+\{[^}]*\}'
all_entries = re.findall(entry_pattern, bib_content, re.DOTALL)


193

In [51]:
unique_entries = []
for citaion_key in unique_citations:
    for entry in all_entries:
        if citaion_key.lower() in entry.lower():
            unique_entries.append(entry)

unique_entries

['@techreport{ARUP2019HEALTHFOOTPRINT,\n    title = {{HEALTH CARE’S CLIMATE FOOTPRINT}',
 '@techreport{Berenjian2019LearningTechnology,\n    title = {{Learning Materials in Biosciences Essentials in Fermentation Technology}',
 '@misc{penVmedicinDK,\n    title = {{Phenoxymethylpenicillinkalium "EQL Pharma" - information til sundhedsfaglige - Medicin.dk}',
 '@article{Baehr2023LifePractices,\n    title = {{Life cycle assessment of pharmaceutical and clinical packaging required for medication administration practices}',
 '@misc{healthcareLCA,\n    title = {{HealthcareLCA database}',
 '@article{Drew2022HealthcareLCA:Assessments,\n    title = {{HealthcareLCA: an open-access living database of health-care environmental impact assessments}',
 '@techreport{Rowe2009HandbookExcipients,\n    title = {{Handbook of Pharmaceutical Excipients}',
 '@misc{EuropeanCentreforDiseasePreventionandControlFactsResistance,\n    title = {{Facts about antimicrobial resistance}',
 '@misc{Armstrong2022ChinaMarket,\